# QCoDeS Example with Yokogawa GS200/GS210

In [1]:
import numpy as np
import time 

import qcodes
from qcodes.instrument_drivers.yokogawa.GS200 import GS200

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


In [2]:
gs = GS200("gs200", 'GPIB0::1::INSTR', terminator="\n")

Connected to: YOKOGAWA GS210 (serial:91T928108, firmware:2.02) in 0.08s


Lets do some sanity checks 

In [3]:
gs.output("off")
gs.source_mode("VOLT")
gs.auto_range(False)
gs.voltage(0)

In [4]:
gs.auto_range(True)

Try to put 1 V on the output 

In [5]:
gs.voltage(1)

The instrument indeed goes to 1 V output. Next, assert that an error is raised if we try to get/set the current 

In [6]:
try:
    gs.current(0.001)  # Set a current
    print("Something has gone wrong")
    assert(False)
except: 
    print("exception correctly raised")

try:
    gs.current() # Get the current
    print("Something has gone wrong")
    assert(False)
except: 
    print("exception correctly raised")

exception correctly raised
exception correctly raised


In [7]:
gs.voltage(0)

The instrument indeed goes back to 0 V output 

In [8]:
gs.source_mode("CURR")

Instrument does indeed go to current source mode

In [9]:
gs.current(0.001)

The instrument does indeed show 1mA output. Note that because output is off, no actual current is flowing

In [10]:
gs.output()

'off'

The instrument is indeed in off mode

Put a 10kOhm resistor over the output and verify that the voltage across the resistor is 1E-3 * 10E3 = 10 V. 

In [11]:
gs.voltage_limit(30)
gs.output("on")

We verify that the voltage on the multmeter is indeed 10V. Let's verify that if we put a 5 volt voltage limit the voltage does indeed drop to 5 V

In [12]:
gs.voltage_limit(5)

We see that the voltage on the multimeter is indeed 5V 

In [13]:
gs.current(0.00)

The multimeter reads 0 mA again and the voltage across the resistor is 0 V 

In [14]:
gs.output("off")

The instrument turns output off. Next assert that exceptions are raised if we try to get/set the voltage while still being in current mode

In [15]:
try:
    gs.volatge(0.0)  # Set a voltage
    print("Something has gone wrong")
    assert(False)
except: 
    print("exception correctly raised")

try:
    gs.voltage() # Get the voltage
    print("Something has gone wrong")
    assert(False)
except: 
    print("exception correctly raised")

exception correctly raised
exception correctly raised


Next, test that we get an exception if we turn off auto ranging and deliberately set the voltage outside of the chosen range

In [16]:
gs.auto_range(False)
gs.source_mode("VOLT")
gs.range(10E-3)

try:
     gs.voltage(1.5)
except ValueError as exp:
    print("exception correctly raised")

exception correctly raised


Verify that when auto range is true, we can set voltages without being worried about ranges

In [17]:
gs.source_mode("VOLT")
gs.auto_range(True)
gs.voltage(1.5)

We verify that we can now set the output to 1.5V without problems. Let see if ramping from 0 to 5 volts with steps of 0.5 works. The delay between the steps shall be 1 second, so the entire ramp should take 10 seconds 

In [18]:
tb = time.time()
gs.voltage(0)
gs.ramp_voltage(5, 0.5, 1)
te = time.time()

print("This took {:.3} seconds".format(te - tb))

This took 10.0 seconds


Now try to ramp the current while still in volt mode. This should raise an error

In [19]:
try:
    gs.ramp_current(0, 1E-3, 1)
    print("Something has gone wrong")
    assert(False)
except:
    print("exception correctly raised")

exception correctly raised


In [20]:
gs.voltage_range()

10.0

In [21]:
try:
    gs.current_range()
    print("Something has gone wrong")
    assert(False)
except:
    print("exception correctly raised")

exception correctly raised


In [22]:
gs.source_mode("CURR")

In [23]:
gs.current_range()

0.001

In [24]:
gs.current(0)

In [25]:
gs.auto_range()

True